# Example to use ProjectQ to run algorithms on Quantum Inspire

Copyright 2018 QuTech Delft. Licensed under the Apache License, Version 2.0.

For more information on Quantum Inspire, see https://www.quantum-inspire.com/.
For more information on ProjectQ, see https://github.com/ProjectQ-Framework/ProjectQ.

In [1]:
import logging
import projectq
from projectq import MainEngine  # import the main compiler engine
from projectq.ops import X, Y, Z, H, Measure,CNOT, Swap, All, CZ  # import the operations we want to perform (Hadamard and measurement)
from projectq.setups import linear
from projectq.setups import restrictedgateset

import quantuminspire.projectq.backend_qx

from getpass import getpass
from coreapi.auth import BasicAuthentication
from quantuminspire.api import QuantumInspireAPI

In [2]:
if 'password' not in vars().keys():
    print('Enter email')
    username = input()
    print('Enter password')
    password = getpass()

auth = BasicAuthentication(username, password)
logger=logging.getLogger('quantum_inspire_api'); logger.setLevel(logging.INFO)
qi = QuantumInspireAPI(r'https://dev.quantum-inspire.com/api/', authentication=auth, logger=logger)
backend_type = qi.get_default_backend_type()

Enter email
gmc@metro.cx
Enter password
········


## Execute algorithm on QX simulator

We create an algorithm to entangle qubit 0 and qubit 4.

In [3]:
qi_backend=quantuminspire.projectq.backend_qx.QIBackend(quantum_inspire_api=qi, backend=backend_type)

engine_list=[projectq.cengines.ManualMapper(lambda ii: ii)]

eng = MainEngine(backend=qi_backend,engine_list=engine_list )  # create a default compiler (the back-end is a simulator)

qubits=eng.allocate_qureg(5)
q1 = qubits[0]
q2 = qubits[-1]

H | q1  # apply a Hadamard gate
CNOT | (q1, q2)
All(Measure) | qubits  # measure the qubits

eng.flush()  # flush all gates (and execute measurements)
print("Measured {}".format( ','.join([ str(int(q)) for q in qubits]) ))  # co

print('Probabilities: %s' % (qi_backend.get_probabilities(qubits),) )

Measured 1,0,0,0,1
Probabilities: {'00000': 0.49609375, '10001': 0.50390625}


The result is as expected: about half of the results is split between 0 and 1 on qubit 0 and 4. The QASM generated by the backend is fairly simple.

In [4]:
print(qi_backend.cqasm())

version 1.0
# generated by Quantum Inspire <class 'quantuminspire.projectq.backend_qx.QIBackend'> class
qubits 5


h q[0]
CNOT q[0], q[4]
measure q[0] 
measure q[1] 
measure q[2] 
measure q[3] 
measure q[4] 


## Simulate a spin-qubit array

On a spin-qubit array we have limited connectivity and also a limited set of gates available. With ProjectQ we can handle these cases by adding specific compiler engines.
Our engine lists is generated by the  `projectq.setups.linear` module.

In [9]:
engine_list = linear.get_engine_list(num_qubits=5, one_qubit_gates='any', two_qubit_gates=(CZ,))

In [10]:
eng = MainEngine(backend=qi_backend,engine_list=engine_list )  # create a default compiler (the back-end is a simulator)

qubits=eng.allocate_qureg(5)
q1 = qubits[0]
q2 = qubits[-1]

H | q1  # apply a Hadamard gate
CNOT | (q1, q2)
All(Measure) | qubits  # measure the qubits

eng.flush()  # flush all gates (and execute measurements)
print("Measured {}".format( ','.join([ str(int(q)) for q in qubits]) ))  # co

print('Probabilities: %s' % (qi_backend.get_probabilities(qubits),) )

Measured 0,0
Probabilities: {'00': 0.48828125, '11': 0.51171875}


The result is the same, but if we look at the QASM generated there is quite a difference. The CNOT gate was replaced by a CZ gate with some single qubit opertations. Also the qubits 0 and 4 have been mapped to neighboring qubits.

In [7]:
print(qi_backend.cqasm())

version 1.0
# generated by Quantum Inspire <class 'quantuminspire.projectq.backend_qx.QIBackend'> class
qubits 5


h q[0]
h q[1]
CZ q[0], q[1]
h q[1]
measure q[0] 
measure q[2] 
measure q[3] 
measure q[4] 
measure q[1] 


In [8]:
current_mapping = eng.mapper.current_mapping
for l, p in current_mapping.items():
    print('mapping logical qubit %d to physical qubit %d' % (l, p))

mapping logical qubit 0 to physical qubit 0
mapping logical qubit 4 to physical qubit 1
mapping logical qubit 1 to physical qubit 2
mapping logical qubit 2 to physical qubit 3
mapping logical qubit 3 to physical qubit 4
